In [ ]:
!pip uninstall -y tensorflow tensorflow-intel tensorflow-cpu keras ml-dtypes


Found existing installation: tensorflow 2.15.0
Uninstalling tensorflow-2.15.0:
  Successfully uninstalled tensorflow-2.15.0
Found existing installation: keras 2.15.0
Uninstalling keras-2.15.0:
  Successfully uninstalled keras-2.15.0
Found existing installation: ml-dtypes 0.2.0
Uninstalling ml-dtypes-0.2.0:
  Successfully uninstalled ml-dtypes-0.2.0


In [ ]:
!pip show tensorflow


In [ ]:
!pip install --quiet torch torchvision torchaudio opencv-python pandas scikit-learn


In [ ]:
import torch, numpy as np
print("Torch:", torch.__version__)
print("NumPy:", np.__version__)


Torch: 2.5.1+cu124
NumPy: 2.2.6


In [ ]:
import os, json, random, math, cv2
from pathlib import Path
import numpy as np
import pandas as pd
from datetime import datetime
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import MinMaxScaler

BASE = Path.home() / "Video_analysis+IOT"
TRAIN_VID_DIR = BASE / "training_videos"
TEST_VID_DIR = BASE / "testing_videos"
IOT_DIR = BASE / "iot_data"
OUTPUT_DIR = BASE / "outputs"

for p in (TRAIN_VID_DIR, TEST_VID_DIR, IOT_DIR, OUTPUT_DIR):
    p.mkdir(parents=True, exist_ok=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("✅ Using device:", device)


✅ Using device: cuda


In [ ]:
def load_videos_as_sequences(folder, seq_len=10, resize=(64,64)):
    all_seqs = []
    for vid_file in sorted(folder.glob("*.avi")):
        cap = cv2.VideoCapture(str(vid_file))
        frames = []
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.resize(frame, resize)
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            gray = gray.astype(np.float32) / 255.0
            frames.append(gray)
        cap.release()
        frames = np.array(frames)
        if len(frames) < seq_len:
            continue
        for i in range(0, len(frames) - seq_len):
            seq = frames[i:i+seq_len]
            all_seqs.append(seq)
    if len(all_seqs) == 0:
        print("⚠️ No videos found or too short. Please ensure .avi files exist in:", folder)
        return np.zeros((0, 1, seq_len, resize[0], resize[1]), dtype=np.float32)
    all_seqs = np.array(all_seqs)
    all_seqs = all_seqs[:, np.newaxis, :, :, :]  # (N,1,T,H,W)
    print(f"🎞️ Loaded {len(all_seqs)} sequences from {folder}")
    return all_seqs

train_data = load_videos_as_sequences(TRAIN_VID_DIR)
test_data = load_videos_as_sequences(TEST_VID_DIR)


🎞️ Loaded 15168 sequences from /home/jovyan/Video_analysis+IOT/training_videos
🎞️ Loaded 15114 sequences from /home/jovyan/Video_analysis+IOT/testing_videos


In [ ]:
class VideoDataset(Dataset):
    def __init__(self, arr): self.x = torch.from_numpy(arr).float()
    def __len__(self): return len(self.x)
    def __getitem__(self, idx): return self.x[idx]

class Conv3DAE(nn.Module):
    def __init__(self):
        super().__init__()
        self.enc = nn.Sequential(
            nn.Conv3d(1, 16, 3, padding=1), nn.ReLU(),
            nn.MaxPool3d((1,2,2)),
            nn.Conv3d(16, 32, 3, padding=1), nn.ReLU(),
            nn.MaxPool3d((1,2,2))
        )
        self.dec = nn.Sequential(
            nn.ConvTranspose3d(32,16,kernel_size=(1,2,2),stride=(1,2,2)), nn.ReLU(),
            nn.ConvTranspose3d(16,1,kernel_size=(1,2,2),stride=(1,2,2)), nn.Sigmoid()
        )
    def forward(self, x):
        return self.dec(self.enc(x))


In [ ]:
#Cell 4 — Train Video Autoencoder
if len(train_data) == 0:
    print("⚠️ No training data, please add videos to training_videos/.")
else:
    train_loader = DataLoader(VideoDataset(train_data), batch_size=8, shuffle=True)
    model = Conv3DAE().to(device)
    opt = torch.optim.Adam(model.parameters(), lr=1e-3)
    loss_fn = nn.MSELoss()

    print("🚀 Training video model...")
    for epoch in range(20):
        total = 0
        for x in train_loader:
            x = x.to(device)
            out = model(x)
            loss = loss_fn(out, x)
            opt.zero_grad(); loss.backward(); opt.step()
            total += loss.item() * x.size(0)
        print(f"Epoch {epoch+1} | Loss: {total/len(train_loader.dataset):.6f}")

    torch.save(model.state_dict(), str(OUTPUT_DIR/"video_conv3d_ae.pth"))
    print("💾 Saved video model to:", OUTPUT_DIR/"video_conv3d_ae.pth")


🚀 Training video model...
Epoch 1 | Loss: 0.005602
Epoch 2 | Loss: 0.002572
Epoch 3 | Loss: 0.002180
Epoch 4 | Loss: 0.001974
Epoch 5 | Loss: 0.001845
Epoch 6 | Loss: 0.001760
Epoch 7 | Loss: 0.001698
Epoch 8 | Loss: 0.001649
Epoch 9 | Loss: 0.001610
Epoch 10 | Loss: 0.001575
Epoch 11 | Loss: 0.001544
Epoch 12 | Loss: 0.001520
Epoch 13 | Loss: 0.001495
Epoch 14 | Loss: 0.001474
Epoch 15 | Loss: 0.001455
Epoch 16 | Loss: 0.001437
Epoch 17 | Loss: 0.001423
Epoch 18 | Loss: 0.001410
Epoch 19 | Loss: 0.001397
Epoch 20 | Loss: 0.001387
💾 Saved video model to: /home/jovyan/Video_analysis+IOT/outputs/video_conv3d_ae.pth


In [ ]:
#Cell 5 — Detect Video Anomalies
if len(test_data) == 0:
    print("⚠️ No test data found. Please add videos to testing_videos/.")
else:
    test_loader = DataLoader(VideoDataset(test_data), batch_size=8, shuffle=False)
    model.eval()
    errors = []
    with torch.no_grad():
        for batch in test_loader:
            batch = batch.to(device)
            out = model(batch)
            mse = torch.mean((out - batch)**2, dim=[1,2,3,4]).cpu().numpy()
            errors.extend(mse.tolist())

    errors = np.array(errors)
    thresh = float(errors.mean() + 3*errors.std())
    anomalies = [{"timestamp": str(datetime.now()), "index": int(i),
                  "recon_error": float(e), "anomaly_type": "Crowd Movement"}
                 for i, e in enumerate(errors) if e > thresh]

    with open(OUTPUT_DIR/"video_anomalies.json", "w") as f:
        json.dump(anomalies, f, indent=4)

    print(f" {len(anomalies)} anomalies detected. Saved -> video_anomalies.json")


 306 anomalies detected. Saved -> video_anomalies.json


In [ ]:
#Cell 6 — Simulate IoT Data
def simulate_iot_data(days=2, freq='1min', anomaly_prob=0.02):
    timestamps = pd.date_range(datetime.now(), periods=int(24*60*days), freq=freq)
    df = pd.DataFrame({
        "timestamp": timestamps,
        "temperature": 25 + np.random.normal(0, 0.4, len(timestamps)),
        "humidity": 50 + np.random.normal(0, 1, len(timestamps)),
        "vibration": 0.03 + np.random.normal(0, 0.01, len(timestamps))
    })
    for i in range(len(df)):
        if random.random() < anomaly_prob:
            col = random.choice(["temperature", "humidity", "vibration"])
            df.at[i, col] += random.choice([5, -5, 7, -7])
    df.to_csv(IOT_DIR/"iot_train.csv", index=False)
    print("IoT simulated:", IOT_DIR/"iot_train.csv")
    return df

iot_df = simulate_iot_data()


IoT simulated: /home/jovyan/Video_analysis+IOT/iot_data/iot_train.csv


In [ ]:
#Cell 7 — Train IoT LSTM Autoencoder
from torch.utils.data import TensorDataset
scaler = MinMaxScaler()
scaled = scaler.fit_transform(iot_df[["temperature","humidity","vibration"]])

SEQ = 30
X = np.array([scaled[i:i+SEQ] for i in range(len(scaled)-SEQ)])
X = torch.tensor(X).float()

train_loader_iot = DataLoader(TensorDataset(X), batch_size=64, shuffle=True)

class LSTM_AE(nn.Module):
    def __init__(self, n_feat=3, hidden=64):
        super().__init__()
        self.encoder = nn.LSTM(n_feat, hidden, batch_first=True)
        self.decoder = nn.LSTM(hidden, hidden, batch_first=True)
        self.fc = nn.Linear(hidden, n_feat)
    def forward(self, x):
        _, (h, _) = self.encoder(x)
        rep = h.repeat(x.size(1), 1, 1).transpose(0,1)
        out, _ = self.decoder(rep)
        return self.fc(out)

iot_model = LSTM_AE().to(device)
opt = torch.optim.Adam(iot_model.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()
for ep in range(10):
    tot = 0
    for b in train_loader_iot:
        b = b[0].to(device)
        out = iot_model(b)
        loss = loss_fn(out, b)
        opt.zero_grad()
        loss.backward()
        opt.step()
        tot += loss.item() * b.size(0)
    print(f"IoT Epoch {ep+1} Loss: {tot/len(train_loader_iot.dataset):.6f}")


torch.save(iot_model.state_dict(), str(OUTPUT_DIR/"iot_lstm_ae.pth"))
print("Saved IoT model:", OUTPUT_DIR/"iot_lstm_ae.pth")


IoT Epoch 1 Loss: 0.039735
IoT Epoch 2 Loss: 0.006249
IoT Epoch 3 Loss: 0.004376
IoT Epoch 4 Loss: 0.003308
IoT Epoch 5 Loss: 0.003197
IoT Epoch 6 Loss: 0.003186
IoT Epoch 7 Loss: 0.003187
IoT Epoch 8 Loss: 0.003185
IoT Epoch 9 Loss: 0.003186
IoT Epoch 10 Loss: 0.003183
Saved IoT model: /home/jovyan/Video_analysis+IOT/outputs/iot_lstm_ae.pth


In [ ]:
#Cell 8 — IoT Anomaly Detection
recon = []
iot_model.eval()
recon = []
with torch.no_grad():
    for b in train_loader_iot:
        b = b[0].to(device)
        out = iot_model(b)
        mse = torch.mean((out - b)**2, dim=(1,2)).cpu().numpy()
        recon.extend(mse)

recon = np.array(recon)
thr = float(recon.mean() + 3*recon.std())

iot_anomalies = [{"timestamp": str(iot_df.iloc[i+SEQ]["timestamp"]),
                  "recon_error": float(recon[i]),
                  "anomaly_type": "IoT Sensor"}
                 for i in range(len(recon)) if recon[i] > thr]

with open(OUTPUT_DIR/"iot_anomalies.json", "w") as f:
    json.dump(iot_anomalies, f, indent=4)

print(f" {len(iot_anomalies)} IoT anomalies detected. Saved -> iot_anomalies.json")


 32 IoT anomalies detected. Saved -> iot_anomalies.json
